In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [7]:
# """
# density
# """
# directory = "data/log/density/10runs0-0.2/"
# # for filename in os.listdir(directory):
# #     print(filename)

# filename = "density=0.01.i=3.csv"
# df = pd.read_csv(f'data/log/density/10runs0-0.2/{filename}')

# for column in df.columns:
#     if 'gate' in column:
#         df = df.drop(columns=column)

# orig_dist = df.filter(regex='distance')
# orig_dist = list(orig_dist.iloc[1])

# # for column in df.columns:
# #     if 'distance' in column:
# #         df = df.drop(columns=column)

# df.columns

# nans = []
# # df['agent_0_x'].isna().idxmax(1)
# for i in range(1, len(df.columns), 3):
#     # start with agent_0_x with steps of 3
#     first_nan = df[df.columns[i]].isna().idxmax(1)
#     nans.append(first_nan)
    
# nans[nans.index(0)] = df.timestep.iat[-1]

# print(nans)
# print(orig_dist)

# # outflux = max timestep / n agents
# # average velocity
# # per agent: orig dist / timestep nans
# # voor systeem: average velocity van alle agents
# # nog een velocity, als een agent bij andere agents staat of die dan langzamer gaat
# # dus velocity aan de hand van hoeveel agents in zn buurt zijn

# # over dataframe per x rows, de velocity over de duur van de run meten.

In [8]:
import numba as nb

@nb.njit(fastmath=True)
def dist(pos1, pos2):
    x = np.array(pos1) - np.array(pos2)
    return np.sqrt(x.dot(x))

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

densities = np.arange(0.01, 0.20, 0.01)
iterations = range(0,10)

# Velocities, leavetime/(startingdistance)
# Outflux, (total sim time)/(agents at start)
# Active average distance between agents; pairwise distances
# kijk voor ierede agent elke 2 timesteps wat zijn gemiddelde afstand is tot andere agents

# Active average Velocity: (position change)/(20 timesteps)

### Output: PER iteratie
# Average Velocity
# Outlfux
# Leave time from agents (nan list)
# Original_distance

"""
voor alle densities 1 rij in een dataframe, met min max en avg

bram?
voor alle densities 1 df die van alle iteraties de avg distance heeft per 20 timesteps
"""

outfluxes = [[] for _ in densities]
avg_velocities = [[] for _ in densities]

for d, density_num in zip(densities, range(len(densities))):
    # columns = ['timestep']
    # columns.extend([f'iteration_{i}_avg_dist_timestep' for i in range(10)])
    distance_df = pd.DataFrame()

    print(distance_df)

    for iteration in iterations:
        df = pd.read_csv(f'../data/log2/density/10runs0-0.2/density={d:.2f}.i={iteration}.csv')
        
        gates_excluded = [x for x in df.columns if "gate" not in x]
        
        orig_dist = df[gates_excluded].filter(regex='distance')
        orig_dist = list(orig_dist.iloc[1])
        
        agent_locations = [x for x in gates_excluded if "orig_distance" not in x]
        df = df[agent_locations]
        
        nans = []
        for column in range(1,len(df.columns),2):
            first_nan = df[df.columns[column]].isna().idxmax(1)
            nans.append(first_nan)
    
        nans[nans.index(0)] = df.timestep.iat[-1]

        outflux = (len(agent_locations)/2) / max(nans)
        outfluxes[density_num].append(outflux)

        average_velocity = np.array(orig_dist) / np.array(nans)
        avg_velocities[density_num].append(np.nanmean(average_velocity))

        steps = 20
        step = 0

        # every x steps add average distance of all agents
        avg_dist_timestep = []
        timesteps = []
        avg_velo_timestep = []

        # so now for every x steps we get an average distance for one agent to the others
        for index, row in df.iterrows():
            if step % steps == 0:
                average_distances = []

                # avg distance
                for j in range(1, len(row), 2):
                    distance = 0
                    amount_of_distances = 0

                    agent_x = row[j]
                    agent_y = row[j+1]
                    if np.isnan(agent_x) or np.isnan(agent_y):
                        continue

                    for k in range(1, len(row), 2):
                        if row[k] == agent_x or np.isnan(row[k]) or np.isnan(row[k+1]):
                            continue
                        
                        amount_of_distances += 1
                        pos1 = (agent_x, agent_y)
                        pos2 = (row[k], row[k+1])
                        distance += dist(pos1, pos2)
                    
                    if amount_of_distances > 0:
                        average_distance = distance / amount_of_distances
                        average_distances.append(average_distance)   
                
                if len(average_distances) > 0:
                    timesteps.append(step)
                    avg_dist_timestep.append(np.nanmean(average_distances))

                # avg velocity
                velocities = []
                if step >= 20:

                    for q in range(1, len(row), 2):
                        current_x = row[q]
                        current_y = row[q+1]
                        old_x = prev_row[q]
                        old_y = prev_row[q+1]

                        if np.isnan(current_x) or np.isnan(current_y) or np.isnan(old_x) or np.isnan(old_y):
                            continue

                        pos1 = (current_x, current_y)
                        pos2 = (old_x, old_y)

                        avg_velo = dist(pos1, pos2) / steps
                        velocities.append(avg_velo)
                    
                avg_velo_timestep.append(np.nanmean(velocities)) if len(velocities) != 0 else avg_velo_timestep.append(np.nan)

                prev_row = row

            step += 1
        # if len(timesteps) > len(distance_df['timestep']):
        df5 = pd.DataFrame({'timestep':timesteps})
        df6 = pd.DataFrame({f"iteration_{iteration}_avg_dist_timestep":avg_dist_timestep})
        df7 = pd.DataFrame({f"iteration_{iteration}_avg_velo_timestep":avg_velo_timestep})
        distance_df = pd.concat([distance_df, df5, df6, df7], axis=1)
        # distance_df.columns=columns
        # distance_df[f'iteration_{i}_avg_dist_timestep'] = pd.Series(avg_dist_timestep)



    break


Empty DataFrame
Columns: []
Index: []


In [12]:
distance_df = distance_df.loc[:,~distance_df.columns.duplicated()]
# distance_df